## Imports

In [ ]:
import pandas as pd
from pandas import DataFrame, Series

import numpy as np

# import ast
from typing import Literal

## AHP

In [2]:
relations_BM = [
    [1,     3,     2/3,   9,     7,     9],
    [1/3,   1,     1/3,   3,     3,     5],
    [2/3,   3,     1,     7,     5,     7],
    [1/9,   1/9,   1/7,   1,     1,     3],
    [1/7,   1/7,   1/5,   1,     1,     3],
    [1/9,   1/5,   1/7,   1/3,   1/3,   1]
]

relations_PB = [
    [1,     3,     2/3,   9,     7,     9],
    [1/3,   1,     1/3,   3,     3,     5],
    [2/3,   3,     1,     7,     5,     7],
    [1/9,   1/9,   1/7,   1,     1,     3],
    [1/9,   1/5,   1/7,   1/3,   1/3,   1], 
    [1/7,   1/7,   1/5,   1,     1,     3]    
]

ordered_columns = [
    'HLA_match',
    'CMV_serostatus',
    'donor_age_group',
    'gender_match',
    'ABO_match',
    'expected_survival_time'
]

In [ ]:
def compute_weights(data_AHP: DataFrame):
    column_sums = data_AHP.sum(axis=0)

    data_AHP_normalized = data_AHP / column_sums

    weights = data_AHP_normalized.mean(axis=1)

    return np.array(weights)

In [ ]:
def get_criteria_weights_AHP(stem_cell_source: Literal["bone marrow", "pheripheral blood"]):
     if stem_cell_source == "bone marrow":
          relation_matrix = relations_BM
     else:
          relation_matrix = relations_PB
         
     data_AHP = DataFrame(relation_matrix, index=ordered_columns, columns=ordered_columns)
    
     weights = compute_weights(data_AHP)

     return weights

## TOPSIS

In [ ]:
def normalize_matrix(matrix):
    matrix_rows = len(matrix)
    matrix_columns = len(matrix[0])
    column_sums= [0] * matrix_columns

    for j in range(matrix_columns):
        for i in range(matrix_rows):
            column_sums[j] += matrix[i][j] ** 2
    column_sums = [value ** 0.5 for value in column_sums]

    normalized_matrix = []
    for i in range(matrix_rows):
        normalized_matrix_rows = []
        for j in range(matrix_columns):
            normalized_matrix_rows.append(matrix[i][j]/column_sums[j])
        normalized_matrix.append(normalized_matrix_rows)
    
    return normalized_matrix

In [ ]:
def weight_matrix(weights, matrix_normalized):
    rows_normalized = len(matrix_normalized)
    cols_normalized = len(matrix_normalized[0])
    
    matrix_weighted = []
    for i in range(rows_normalized):
        matrix_weighted_rows = []
        for j in range(cols_normalized):
            matrix_weighted_rows.append(matrix_normalized [i][j]* weights[j])
        matrix_weighted.append(matrix_weighted_rows)

    return matrix_weighted

In [ ]:
def get_deviation_from_ideal_col_TOPSIS(data_criteria_encoded: DataFrame, stem_cell_source: Literal["bone marrow", "pheripheral blood"]):
    ordered_columns = [
        'HLA_match',
        'CMV_serostatus',
        'donor_age_group',
        'gender_match',
        'ABO_match',
        'expected_survival_time'
    ]

    data_criteria_encoded = data_criteria_encoded.loc[:, ordered_columns].copy()

    # Estrutura do DataFrame necessária para o funcionamento do TOPSIS:
    # |─────────────|───────────|───────────|─────────────────|───────────────────|───────────────|────────────|──────────────────────────|-----------|--------------|
    # |recipient_ID │ donor_ID  │ HLA Match │ CMV Serostatus  │  Donor Age Group  │ Gender Match  │ ABO Match  │ Expected Survival Time   │Donor Name |Recipient Name|
    # ├─────────────┼───────────┼───────────┼─────────────────┼───────────────────┼───────────── ─┼────────────┤──────────────────────────┤-----------|--------------|        
    # |    str      │ str       │ int       │ int             │ int               │ int           │ int        │ int                      │str        |str           |
    # └─────────────┴───────────┴───────────┴─────────────────┴───────────────────┴───────────── ─┴────────────┘──────────────────────────┘-----------|--------------|

    impact = np.array([1, -1, 1, 1, 1, 1])

    weights = get_criteria_weights_AHP(stem_cell_source)
    
    criteria_matrix = data_criteria_encoded.to_numpy()

    # Chama as funções auxiliares
    matrix_normalized = normalize_matrix(criteria_matrix)
    matrix_weighted = weight_matrix(weights, matrix_normalized)
    pos_ideal, neg_ideal = ideal_best_worst(w_matrix, impact)
    pos_sep, neg_sep = separation_from_ideal_point(w_matrix, pos_ideal, neg_ideal)
    deviations = similarities_to_PIS(pos_sep, neg_sep)

    deviation_from_ideal_col = Series(deviations, name='deviation_from_ideal')
    return deviation_from_ideal_col

## Testing

In [ ]:
# weights = get_criteria_weights("bone marrow")

array([0.37343966, 0.15024488, 0.32553913, 0.05517793, 0.06195631,
       0.03364209])